# get timezones

In [52]:
from datetime import datetime, timedelta
import pytz
import numpy as np

In [53]:
set(pytz.all_timezones_set) 


{'Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [153]:
user_tz.date()


datetime.date(2021, 10, 25)

In [154]:
sw_tz = user_tz.astimezone(pytz.timezone('Europe/Zurich'))


In [ ]:
# create a dictionary to add to df for plotting

In [ ]:
# alternating timezone & start-end

In [320]:
def inputs_to_rowdicts(user_timezone, input_name, input_timezone, input_hourrange):
    
    start_hour, end_hour = input_hourrange[0], input_hourrange[1]

    user_now = datetime.now(tz=pytz.timezone(user_timezone))
    
    input_now = user_now.astimezone(pytz.timezone(input_timezone))

    input_start = input_now - timedelta(hours = 24)

    times = pd.date_range(input_start, periods=72, freq='1h').tolist()
    
    #keep obs if they are the times of interest
    time_blips = [time for time in times if time.hour == start_hour or time.hour == end_hour]
    
    # fill in missing start/end times
    if time_blips[0].hour == end_hour:
        time_blips.insert(0, np.min(times))
    if time_blips[-1].hour == start_hour:
        time_blips.insert(-1, np.max(times))
    
    # convert into users timezone
    user_timeblips = [time.astimezone(pytz.timezone(user_timezone)) for time in time_blips]
    
    # create dictionary of rows for the inputs
    row_dict_list = []
    for x in range(int(len(user_timeblips)/2)):
              row_dict_list.append({'person':input_name,'start':user_timeblips.pop(0),'end':user_timeblips.pop(0)})
        
    return row_dict_list
    


In [328]:
def input_to_dataframe(inputs):

    timezone_inputs = inputs[::2]

    hourrange_inputs = inputs[1::2]         

    name_inputs = ['You'] + [f"Person {str(i + 1)}" for i in range(int(len(inputs)/2 - 1))]

    user_tz = timezone_inputs[0] 

    all_waketimes = []                 
    for person in zip(name_inputs, timezone_inputs, hourrange_inputs):

        rowdict = inputs_to_rowdicts(user_timezone = user_tz, 
                                    input_name = person[0],
                                    input_timezone = person[1], 
                                    input_hourrange = person[2])

        all_waketimes = all_waketimes + rowdict

    return pd.DataFrame(all_waketimes)

In [324]:
all_waketimes

[{'person': 'You',
  'start': Timestamp('2021-10-26 18:46:49.229943+1100', tz='Australia/Victoria', freq='H'),
  'end': Timestamp('2021-10-26 19:46:49.229943+1100', tz='Australia/Victoria', freq='H')},
 {'person': 'You',
  'start': Timestamp('2021-10-27 18:46:49.229943+1100', tz='Australia/Victoria', freq='H'),
  'end': Timestamp('2021-10-27 19:46:49.229943+1100', tz='Australia/Victoria', freq='H')},
 {'person': 'You',
  'start': Timestamp('2021-10-28 18:46:49.229943+1100', tz='Australia/Victoria', freq='H'),
  'end': Timestamp('2021-10-28 19:46:49.229943+1100', tz='Australia/Victoria', freq='H')},
 {'person': 'Person 1',
  'start': Timestamp('2021-10-26 18:46:49.234854+1100', tz='Australia/Victoria', freq='H'),
  'end': Timestamp('2021-10-26 19:46:49.234854+1100', tz='Australia/Victoria', freq='H')},
 {'person': 'Person 1',
  'start': Timestamp('2021-10-27 18:46:49.234854+1100', tz='Australia/Victoria', freq='H'),
  'end': Timestamp('2021-10-27 19:46:49.234854+1100', tz='Australia/Vic

In [ ]:
# people being compared to user
localFormat = "%Y-%m-%d %H:%M:%S"
timezones = ['America/Los_Angeles', 'Europe/Madrid', 'America/Puerto_Rico']

localDatetime = user_tz.astimezone(pytz.timezone(tz))
print(localDatetime.strftime(localFormat))


In [ ]:
# function to take in time zone and hours and convert


In [42]:

compare1_timezone = 'Europe/Zurich'
    
user_timezone = 'NZ'

user_start = '1996-02-12 08:00:00' 

user_start_time = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")

user_tz = timezone("NZ")
au_dt = utc_dt.astimezone(au_tz)

# format the datetime as expected by the pipeline
formatted_pickup_datetime = user_start_time.strftime("%Y-%m-%d %H:%M:%S UTC")




utcmoment_naive: 2021-10-24 05:51:44.820387
utcmoment:       2021-10-24 05:51:44.820387+00:00
2021-10-23 22:51:44
2021-10-24 07:51:44
2021-10-24 01:51:44


NameError: name 'pickup_datetime' is not defined

# get hours

In [22]:
import pandas as pd

date_today = '1996-02-12 08:00:00'

times = pd.date_range(date_today, periods=24, freq='1h')


times


# create graph

## create label mapping dictionary

In [137]:
ylabs = ['12am'] + [f"{str(x)}am" for x in range(1, 12)] +  ['12pm'] + [f"{str(x)}pm" for x in range(1, 12)]

lab_dict = {k: v for k, v in zip(np.arange(0, 24), ylabs)}

In [138]:
lab_dict

{0: '12am',
 1: '1am',
 2: '2am',
 3: '3am',
 4: '4am',
 5: '5am',
 6: '6am',
 7: '7am',
 8: '8am',
 9: '9am',
 10: '10am',
 11: '11am',
 12: '12pm',
 13: '1pm',
 14: '2pm',
 15: '3pm',
 16: '4pm',
 17: '5pm',
 18: '6pm',
 19: '7pm',
 20: '8pm',
 21: '9pm',
 22: '10pm',
 23: '11pm'}

In [107]:
import plotly.graph_objects as go

people = ['You', 'Person 1']

starts = ['', -8]

ends = [-17, -17]

fig = go.Figure(data=[
    
    go.Bar(x = people, y = ends, base = starts)
])
# Change the bar mode
fig.update_layout(
    barmode='stack',
    yaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(-23, 1),
        ticktext = ylabs[::-1]
))

#fig.update_xaxes(range=[0, 24])
fig.update_yaxes(range=[-23, 1])

fig.show()


In [141]:
date_today = '1996-02-12'

times = pd.date_range(date_today, periods=24, freq='1h')

times



DatetimeIndex(['1996-02-12 00:00:00', '1996-02-12 01:00:00',
               '1996-02-12 02:00:00', '1996-02-12 03:00:00',
               '1996-02-12 04:00:00', '1996-02-12 05:00:00',
               '1996-02-12 06:00:00', '1996-02-12 07:00:00',
               '1996-02-12 08:00:00', '1996-02-12 09:00:00',
               '1996-02-12 10:00:00', '1996-02-12 11:00:00',
               '1996-02-12 12:00:00', '1996-02-12 13:00:00',
               '1996-02-12 14:00:00', '1996-02-12 15:00:00',
               '1996-02-12 16:00:00', '1996-02-12 17:00:00',
               '1996-02-12 18:00:00', '1996-02-12 19:00:00',
               '1996-02-12 20:00:00', '1996-02-12 21:00:00',
               '1996-02-12 22:00:00', '1996-02-12 23:00:00'],
              dtype='datetime64[ns]', freq='H')

In [ ]:
[[8, 17],
[8, 17]]

In [145]:
times[0].astimezone(pytz.timezone('Europe/Zurich'))

TypeError: Cannot convert tz-naive Timestamp, use tz_localize to localize

In [128]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Task="You", Start='1996-02-12 08:00:00', Finish='1996-02-12 17:00:00'),
    dict(Task="Person 1", Start='1996-02-12 08:00:00', Finish = '1996-02-12 17:00:00')
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
inputs = ['Australia/Victoria',
          [18, 19],
          'Europe/Zurich',
          [9, 10],
          'NZ',
          [8,17]]

In [336]:
df = input_to_dataframe(inputs)

In [339]:
df.person.unique()

array(['You', 'Person 1', 'Person 2'], dtype=object)

In [338]:
fig = px.timeline(df, x_start="start", x_end="end", y="person")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.add_vrect(x0=df.start[0], x1=df.end[0])
fig.show()

# getting limits

In [334]:
def get_limits(user_timezone):

    user_now = datetime.now(tz=pytz.timezone(user_timezone))

    user_start = user_now - timedelta(hours=2)

    times = pd.date_range(user_start, periods=26, freq='1h').tolist()

    return np.min(times), np.max(times)

In [335]:
get_limits("NZ")

(Timestamp('2021-10-27 01:44:04.427163+1300', tz='NZ', freq='H'),
 Timestamp('2021-10-28 02:44:04.427163+1300', tz='NZ', freq='H'))

# getting date overlap